In [28]:
import torch
from matplotlib import pyplot as plt 
import numpy as np
import random

num_inputs = 2
num_features = 1000
true_w = [2,-3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0,1,(num_features,num_inputs)),dtype=torch.float32)
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1] + true_b
labels += torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float32)

print(features[0],labels[0])

tensor([-0.1525, -1.2754]) tensor(8.2324)


## 使用Dataloader读取数据
在前面中，我们自定义了`data_iter`来遍历训练样本，在Pytorch中封装了`torch.utils.data`包进行数据的读取与处理。对于简单的数据集来说，可以直接使用`TensorDataset`包装，然后使用`Dataloader`进行读取。 对于复杂或者无法一次加载到内存中的数据计，可以自定义`Dataset`。

In [29]:
import torch.utils.data as Data 

batch_size = 10
dataset = Data.TensorDataset(features,labels)
data_iter = Data.DataLoader(dataset,batch_size=batch_size,shuffle=True)

for X,y in data_iter:
    print(X,y)
    break

tensor([[ 0.2746,  0.2303],
        [ 0.7015,  0.6191],
        [-0.9822,  1.3436],
        [ 1.1595, -0.0456],
        [-0.0541,  1.0804],
        [-0.3530, -0.2207],
        [-0.5002,  0.2155],
        [-1.1924,  0.6273],
        [-0.4808,  1.0043],
        [-0.5958,  0.3573]]) tensor([ 3.9597,  3.5205, -2.3312,  6.6775,  0.4075,  4.2445,  2.4669, -0.3224,
        -0.1832,  1.7917])


## 定义网络模型

这里自定义一个网络层来实现线性回归。 首先，导入`torch.nn`模块，nn表示**neural networks**,该模块定义了大量神经网络的层。`nn`的核心数据结构为`Module`，是一个抽象的概念，**即可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的的网络。

在实际使用中，通常可以继承`nn.Module`，实现自定义的网络/层。一个`nn.Module`实例应该包含一些层以及返回输出结果的前向传播(forward)方法。


In [30]:

import torch.nn as nn
class LinearNet(nn.Module):
    def __init__(self,n_feature):
        super(LinearNet,self).__init__()
        self.linear = nn.Linear(n_feature,1)

    def forward(self,x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net)



LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


可以使用PyTorch提供的容器`Sequential`来方便快捷的搭建网络。`Sequential`是一个有序容器，网络层将按照加入到`Sequential`的顺序被依次添加到计算图中。
```
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs,1)
)
# 写法二
net = nn.Sequential()
net.add_module('linear',nn.Linear(num_inputs,1))
# net.add_module ...

# 写法三
from collection import OrdereDict
net = nn.Sequential(OrdereDict([
    ('linear',nn.Linear(num_inputs,1))
    # ()...
    ]))
```

## 初始化模型参数

### 参数的访问

In [31]:
# 可以调用`parameters()`或者`named_parameters()`来访问网络各个层的参数

for param in net.parameters():
    print(param)
    print(param.data)

print("----- named parameters -----")
for name,param in net.named_parameters():
    print(name)

Parameter containing:
tensor([[ 0.1447, -0.3669]], requires_grad=True)
tensor([[ 0.1447, -0.3669]])
Parameter containing:
tensor([-0.5967], requires_grad=True)
tensor([-0.5967])
----- named parameters -----
linear.weight
linear.bias


`parameters()`返回包含模型所有参数的迭代器。每个模型参数包括两部分：`data`参数的值和`requires_grad`指示是否自动求导。在上一个实现中，自定义参数的时候，需要手动的制定其`requreid_grad`属性.
```
w = torch.tensor(np.random.normal(0,0.01,(num_inputs,1)),dtype=torch.float32)
b = torch.zeros(1,dtype=torch.float32)

w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)
```
`parameters()`通常作为参数传递给`optimizer`使用。_

`name_parameters()`不仅仅返回模型参数的，还返回该参数的名称，例如：`linear.weight`。 这样在微调中有选择的加载预训练模型就比较方便，可以根据其名称有选择的加载或者不记载。


### 参数初始化
模型训练前要对其参数进行初始化，对于bias参数可以将其设置为0。对于权重则不能这么简单的处理，PyTorch的`init`模块中提供了多种参数初始化方法。下面使用`init.normal_`将权重初始化为均值为0，标准差为0.01的正态分布

In [32]:
from torch.nn import init

for name,param in net.named_parameters():
    if "weight" in name:
        init.normal_(param.data,mean=0,std=0.01)
    elif "bias" in name:
        init.constant_(param.data,val=0)

for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0013, -0.0009]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


## 损失函数

`nn`模块中提供了各种损失函数，损失函数可以作为神经网络中的一个**特殊的层**,这些损失函数也是`nn.Module`的子类

In [33]:
loss = nn.MSELoss()

## 优化方法
`torch.optim`提供了很多常用的优化算法比如SGD,Adam以及RMSPro等。 下面使用SGD并将其学习率指定为0.03.

In [34]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(),lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


还可以为不同的网络层设置不同的学习率，这在finetune的时候是非常有用的。
```
optimizer = optim.SGD([
    {'params':net.subnet1.parameters()}, # 没有指定learningrate就是用默认的学习率
    {'params':net.subnet2.parameters(),lr=0.01}
],lr=0.03)
```

在有些情况下，不想将学习率固定为常数，有两种方法可以对其进行调整：
- 修改`optimizer.param_groups`中对应的学习率
- 新建一个优化器，由于`optimizer`是轻量级的，构建的开销很小，但是这样做对使用动量的优化器（如Adam），会丢失动量信息，造成损失函数的震荡。_
```
for parm_group in optimizer.param_groups:
    param_group["lr"] *= 0.01 # 学习率为之前的0.01
```

## 模型训练

进行模型训练时，通过调用`optimizer`的`step`函数进行迭代

In [36]:
num_epochs = 3
for epoch in range(0,num_epochs):
    for X,y in data_iter:
        output= net(X)
        l = loss(output,y.view(-1,1))
        optimizer.zero_grad() # 梯度清零
        l.backward() # 反向传播
        optimizer.step()

    print("epoch %d,loss:%f" %(epoch + 1,l.item()))

# 输出参数值
for name,param in net.named_parameters():
    print(name)
    print(param.data)



epoch 1,loss:0.000052
epoch 2,loss:0.000066
epoch 3,loss:0.000078
linear.weight
tensor([[ 1.9997, -3.4001]])
linear.bias
tensor([4.1989])


## 总结

PyTorch使用流程：

- **训练数据的搜集处理**，对于简单的数据可以直接使用`TensorDataset`。通常数据集无法全部加载到内存中，可以自定义`Dataset`，然后使用`Dataloader`迭代读取。
- **模型的定义**，网络结构的实现。可以自定义`nn.Module`，也可以使用PyTorch提供的容器类`Sequeantial`或者`ModuleList`，依次添加网络层。
- **模型参数初始化**，`torch.init`模块提供了各种样的参数初始化方法。 这里可以使用`parameters()`或者`named_parameters()`来访问网络层的参数。 
- **损失函数定义**， 损失函数可以作为一个特殊层存在于网络中，在`nn`模块提供了各种常见的损失函数。
- **定义优化算法** ，`torch.optim`模块提供了各种常用的优化算法，例如：SGD，Adam以及RMSprop等。PyTorch也提供了一些动态修改参数的机制。
- **训练** 其训练过程可以使用如下代码模板
    ```
    num_epochs = 3
    for epoch in range(0,num_epochs):
        for X,y in data_iter:
            output= net(X)
            l = loss(output,y.view(-1,1))
            optimizer.zero_grad() # 梯度清零
            l.backward() # 反向传播
            optimizer.step()

        print("epoch %d,loss:%f" %(epoch + 1,l.item()))
    ```
    在每个epoch中调用`Dataloader`得到每个batch的训练样本，
    - 使用模型对样本数据进行正向传播`net(x)`，得到当前模型预测结果。 
    - 调用损失函数`loss`计算当前模型的预测结果$\hat{y}$ 和样本标签$y$的误差。
    - 调用`backward`进行反向传播，计算各个节点的梯度
    - `step`更新网络层的各个参数